In [1]:
import pandas as pd

In [2]:
df_usage = pd.read_csv('./usage.csv')
df_no_flex = pd.read_excel('./contracts.xlsx', sheet_name='no flex')
df_hourly_flex = pd.read_excel('./contracts.xlsx', sheet_name='hourly flex')
df_monthly_flex = pd.read_excel('./contracts.xlsx', sheet_name='monthly flex')

In [3]:
df_usage

,Hour,TI(AM/PM),Weekday,Day,Month,Electricity(kwh)
0,3,PM,mon,24,Mar,0.384
1,5,AM,NaN,15,Aug,1.201
2,8,PM,thu,20,Mar,1.523
3,6,PM,NaN,23,Apr,0.424
4,1,AM,fri,19,Dec,0.209
...,...,...,...,...,...,...
8755,1,AM,fri,7,Nov,0.084
8756,6,AM,NaN,20,May,1.027
8757,2,AM,tue,8,Apr,0.052
8758,9,PM,NaN,27,Jan,1.428


In [4]:
# Answers

# Question 1 
df_usage['Electricity(kwh)'].mean().round(3) 
# Answer: b

# Question 2 
usage_in_feb = df_usage[df_usage['Month'].str.lower() == 'feb']
usage_in_feb['Electricity(kwh)'].mean().round(3) 
# Answer: d

# Question 3
weekdays_avg = df_usage.groupby('Weekday')['Electricity(kwh)'].mean()
weekdays_avg.idxmax()
# Answer: Saturday

# Question 4
df_usage['Date'] = df_usage['Day'].astype(str) + '-' + df_usage['Month']
df_usage['DateTime'] = pd.to_datetime(df_usage['Date'] + ' ' + 
                                df_usage['Hour'].astype(str) + ' ' + 
                                df_usage['TI(AM/PM)'], 
                                format='%d-%b %I %p')
df_usage = df_usage.sort_values('DateTime')
df_usage['Rolling_Sum'] = df_usage['Electricity(kwh)'].rolling(window=4).sum()
max_4hour_usage = df_usage['Rolling_Sum'].max()
max_4hour_usage.round(3)
# Answer: a

# Question 5
monthly_elec_usage = df_usage.groupby('Month')['Electricity(kwh)'].sum()
monthly_elec_usage_df = monthly_elec_usage.reset_index()
monthly_elec_usage_df.columns = ['Month', 'Usage']
df_monthly_flex['Month'] = df_monthly_flex['Month'].str.slice(0,3).str.lower()
monthly_elec_usage_df['Month'] = monthly_elec_usage_df['Month'].str.lower()
merged_df = pd.merge(monthly_elec_usage_df, df_monthly_flex, on='Month')
cost_per_month = merged_df['Usage']*merged_df['Rate']
cost_per_month.sum()
# Answer: b

# Question 6
monthflex_cost = cost_per_month.sum() 
monthflex_cost # 1421

noflex_cost = monthly_elec_usage_df['Usage'].sum()*df_no_flex['Rate']
noflex_cost # 1438

df_hourly_flex['Hour'] = df_hourly_flex['Hourly'].str.split(r'[AP]', n=1).str[0]
df_hourly_flex['TI(AM/PM)'] = df_hourly_flex['Hourly'].str.split('u', n=1).str[0].str.extract(r'([a-zA-Z]+)')
df_usage_hourly = df_usage[['Hour', 'TI(AM/PM)', 'Electricity(kwh)']]
df_hourly_flex['Hour'] = df_hourly_flex['Hour'].astype(int)
merged_h = pd.merge(df_hourly_flex, df_usage_hourly, on=['Hour', 'TI(AM/PM)'], how='inner')
cost_per_h = merged_h['Rate']*merged_h['Electricity(kwh)']
hourly_cost = cost_per_h.sum()
hourly_cost #1369
# Answer: c

1369.36063

In [5]:
weekdays_avg = df_usage.groupby('Weekday')['Electricity(kwh)'].mean()
weekdays_avg
# weekdays_avg.idxmax()

Weekday
fri    0.737449
mon    0.740523
sat    0.891982
sun    0.889580
thu    0.785226
tue    0.708859
wed    0.755760
Name: Electricity(kwh), dtype: float64